In [1]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-l.json'); 
var kreativesFeld = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 43  
Gruppenanzahl: 8

In [2]:
var groups = 8;

In [3]:
// 100x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < 100; run++) {
    
    // Calculate groups with monte carlo
    runs.push(kreativesFeld.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

var highscoreMonteCarlo = undefined;

// Run async runs in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

[
  -213.75908210571075,  -208.0998698193113,  -175.3872873637938,
  -136.48926747870667, -153.95377254964845, -48.421564304373405,
  -122.54668243162318, -108.12607843996507, -219.52679847475565,
    -64.7338934311077, -110.03732835436917,  -193.8473669489154,
  -177.74509980172616, -194.84489940677614,  -144.8200782970996,
   -87.40347431547964, -154.11724741565348, -191.70791022353484,
  -225.95023471762738, -161.71748096711264,  -193.8399431868216,
  -204.67133907438364, -182.27365647182933,  -218.2677734494014,
  -177.58724714956622, -139.70767776380384, -173.09486141113155,
  -165.54238144712963, -156.41903559778012, -258.72603105592094,
  -191.21648745031658, -177.96604227042795, -177.20368567465502,
  -192.85721679655808, -143.77715318969183, -159.63391011254163,
  -141.88146469992157,  -55.16189861760532, -143.09537097233465,
  -140.76071258755204, -122.93514039971916, -173.30817829308774,
  -164.58084119974313,  -222.1553404743001, -154.33223048002444,
   -180.4610269167752, 

In [4]:
// 100x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

// Run algorithm x times
for(let run = 0; run < 100; run++) {
    runs.push(kreativesFeld.genetic(input, groups).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

[
  -217.94187742422133, -159.46280665561486, -206.58004689735935,
  -255.72905396034827, -195.44648617343873, -251.48954358813992,
  -237.07865052945476, -229.80056678012727, -225.50383819126137,
  -188.56548964361764,  -171.0400224194179, -128.80291650540676,
  -240.47778113680053, -272.86214553874044, -160.83590413394268,
   -189.1123500063351, -272.08019937895097,  -265.1977047496124,
  -212.25025059764207, -229.04163468698937, -209.93358582163506,
  -148.31323305184364, -219.38535726767196, -155.17004016544752,
  -239.73833570886285, -190.42410292859492, -188.46401771264885,
  -203.52539682243318, -222.45449398093973, -252.65170887872762,
   -265.5839123029999,  -134.6601137613322,  -152.3262274994632,
   -221.6816334648094, -202.60905165293607, -188.25943026897107,
   -245.3461236460427, -255.25208604313394,  -222.0743444887175,
   -186.8024114966134, -105.87165373439711, -251.45069316622897,
  -161.28729125734205, -256.39364920310487, -251.52001786675444,
  -169.67231073540995, 

## Vergleich Algorithmen

In [ ]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

/*
var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);
*/
var hitRateMonteCarlo = scoresMonteCarlo;
var hitRateGenetic = scoresGenetic;

In [ ]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [ ]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();